In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import pickle
import cv2
import time
import glob
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop 
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv3D, MaxPooling3D, Input, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

In [ ]:
df_train = pd.read_csv(r"/content/drive/My Drive/Upenn_Data/training_data.csv") #refer .csv for cross-validation implementation
df_val = pd.read_csv(r"/content/drive/My Drive/Upenn_Data/validation_data.csv")
column = "CPM_RadPath_2019_ID"
class_name = "class"
input_path = r'/content/drive/My Drive/UPenn_Data/miccai2019-data/CPM-RadPath_2019_Training_Data/Radiology'
flair_format = "_flair.nii.gz"
t1_format = "_t1.nii.gz"
t1ce_format = "_t1ce.nii.gz"
t2_format = "_t2.nii.gz"

In [ ]:
train_flair = []
val_flair = []
train_t1 = []
val_t1 = []
train_t1ce = []
val_t1ce = []
train_t2 = []
val_t2 = []
class_train = []
class_val = []

In [ ]:
for i in range(0,208):
    data_path = df_train[column][i]
    
    flair_path = os.path.join(input_path,data_path,data_path+flair_format)
    flair_image = nib.load(flair_path)
    flair = flair_image.get_data()
    flair[flair!=0] = ((((flair[flair!=0] - np.mean(flair[flair!=0]))/ np.std(flair[flair!=0]))+ 10)* 100).astype(dtype=np.int16)
    train_flair.append(flair)
    
    t1_path = os.path.join(input_path,data_path,data_path+t1_format)
    t1_image = nib.load(t1_path)
    t1 = t1_image.get_data()
    t1[t1!=0] = ((((t1[t1!=0] - np.mean(t1[t1!=0]))/ np.std(t1[t1!=0]))+ 10)* 100).astype(dtype=np.int16)
    train_t1.append(t1)
    
    t1ce_path = os.path.join(input_path,data_path,data_path+t1ce_format)
    t1ce_image = nib.load(t1ce_path)
    t1ce = t1ce_image.get_data()
    t1ce[t1ce!=0] = ((((t1ce[t1ce!=0] - np.mean(t1ce[t1ce!=0]))/ np.std(t1ce[t1ce!=0]))+ 10)* 100).astype(dtype=np.int16)
    train_t1ce.append(t1ce)
    
    t2_path = os.path.join(input_path,data_path,data_path+t2_format)
    t2_image = nib.load(t2_path)
    t2 = t2_image.get_data()
    t2[t2!=0] = ((((t2[t2!=0] - np.mean(t2[t2!=0]))/ np.std(t2[t2!=0]))+ 10)* 100).astype(dtype=np.int16)
    train_t2.append(t2)
    
    train_class.append(df_train[class_name][i])

In [ ]:
for i in range(0,52):
    data_path = df_val[column][i]
    
    flair_path = os.path.join(input_path,data_path,data_path+flair_format)
    flair_image = nib.load(flair_path)
    flair = flair_image.get_data()
    flair[flair!=0] = ((((flair[flair!=0] - np.mean(flair[flair!=0]))/ np.std(flair[flair!=0]))+ 10)* 100).astype(dtype=np.int16)
    val_flair.append(flair)
    
    t1_path = os.path.join(input_path,data_path,data_path+t1_format)
    t1_image = nib.load(t1_path)
    t1 = t1_image.get_data()
    t1[t1!=0] = ((((t1[t1!=0] - np.mean(t1[t1!=0]))/ np.std(t1[t1!=0]))+ 10)* 100).astype(dtype=np.int16)
    val_t1.append(t1)
    
    t1ce_path = os.path.join(input_path,data_path,data_path+t1ce_format)
    t1ce_image = nib.load(t1ce_path)
    t1ce = t1ce_image.get_data()
    t1ce[t1ce!=0] = ((((t1ce[t1ce!=0] - np.mean(t1ce[t1ce!=0]))/ np.std(t1ce[t1ce!=0]))+ 10)* 100).astype(dtype=np.int16)
    val_t1ce.append(t1ce)
    
    t2_path = os.path.join(input_path,data_path,data_path+t2_format)
    t2_image = nib.load(t2_path)
    t2 = t2_image.get_data()
    t2[t2!=0] = ((((t2[t2!=0] - np.mean(t2[t2!=0]))/ np.std(t2[t2!=0]))+ 10)* 100).astype(dtype=np.int16)
    val_t2.append(t2)
    
    val_class.append(df_val[class_name][i])

In [ ]:
train_class = []
val_class = []

for i in class_train:
    if i == 'G':
        train_class.append(1)
    else:
        train_class.append(0)

for i in class_val:
    if i == 'G':
        val_class.append(1)
    else:
        val_class.append(0)

train_labels = to_categorical(train_class)
val_labels = to_categorical(val_class)

In [ ]:
train_flair = np.array(train_flair)
val_flair = np.array(val_flair)
train_t1 = np.array(train_t1)
val_t1 = np.array(val_t1)
train_t1ce = np.array(train_t1ce)
val_t1ce = np.array(val_t1ce)
train_t2 = np.array(train_t2)
val_t2 = np.array(val_t2)

train_flair = np.array(train_flair).reshape(208, 80, 96, 64, 1)
val_flair = np.array(val_flair).reshape(52, 80, 96, 64, 1)
train_t1 = np.array(train_t1).reshape(208, 80, 96, 64, 1)
val_t1 = np.array(val_t1).reshape(52, 80, 96, 64, 1)
train_t1ce = np.array(train_t1ce).reshape(208, 80, 96, 64, 1)
val_t1ce = np.array(val_t1ce).reshape(52, 80, 96, 64, 1)
train_t2 = np.array(train_t2).reshape(208, 80, 96, 64, 1)
val_t2 = np.array(val_t2).reshape(52, 80, 96, 64, 1)

In [ ]:
input_shape = Input(shape = (80, 96, 64, 1))

conv1_1 = Conv3D(32, 3, activation = 'relu')(input_shape)
conv1_2 = Conv3D(32, 3, activation = 'relu')(conv1_1)
pool1_1 = MaxPooling3D(pool_size=(3,3,3))(conv1_2)
conv1_3 = Conv3D(32, 3, activation = 'relu')(pool1_1)
conv1_4 = Conv3D(32, 3, activation = 'relu')(conv1_3)
pool1_2 = MaxPooling3D(pool_size=(3,3,3))(conv1_4)
conv1_5 = Conv3D(32, 3, activation = 'relu')(pool1_2)
conv1_6 = Conv3D(32, 3, activation = 'relu')(conv1_5)
pool1_3 = MaxPooling3D(pool_size=(3,3,3))(conv1_6)
flat1 = Flatten()(pool1_3)

conv2_1 = Conv3D(32, 3, activation = 'relu')(input_shape)
conv2_2 = Conv3D(32, 3, activation = 'relu')(conv2_1)
pool2_1 = MaxPooling3D(pool_size=(3,3,3))(conv2_2)
conv2_3 = Conv3D(32, 3, activation = 'relu')(pool2_1)
conv2_4 = Conv3D(32, 3, activation = 'relu')(conv2_3)
pool2_2 = MaxPooling3D(pool_size=(3,3,3))(conv2_4)
conv2_5 = Conv3D(32, 3, activation = 'relu')(pool2_2)
conv2_6 = Conv3D(32, 3, activation = 'relu')(conv2_5)
pool2_3 = MaxPooling3D(pool_size=(3,3,3))(conv2_6)
flat2 = Flatten()(pool2_3)

conv3_1 = Conv3D(32, 3, activation = 'relu')(input_shape)
conv3_2 = Conv3D(32, 3, activation = 'relu')(conv3_1)
pool3_1 = MaxPooling3D(pool_size=(3,3,3))(conv3_2)
conv3_3 = Conv3D(32, 3, activation = 'relu')(pool3_1)
conv3_4 = Conv3D(32, 3, activation = 'relu')(conv3_3)
pool3_2 = MaxPooling3D(pool_size=(3,3,3))(conv3_4)
conv3_5 = Conv3D(32, 3, activation = 'relu')(pool3_2)
conv3_6 = Conv3D(32, 3, activation = 'relu')(conv3_5)
pool3_3 = MaxPooling3D(pool_size=(3,3,3))(conv3_6)
flat3 = Flatten()(pool3_3)

conv4_1 = Conv3D(32, 3, activation = 'relu')(input_shape)
conv4_2 = Conv3D(32, 3, activation = 'relu')(conv4_1)
pool4_1 = MaxPooling3D(pool_size=(3,3,3))(conv4_2)
conv4_3 = Conv3D(32, 3, activation = 'relu')(pool4_1)
conv4_4 = Conv3D(32, 3, activation = 'relu')(conv4_3)
pool4_2 = MaxPooling3D(pool_size=(3,3,3))(conv4_4)
conv4_5 = Conv3D(32, 3, activation = 'relu')(pool4_2)
conv4_6 = Conv3D(32, 3, activation = 'relu')(conv4_5)
pool4_3 = MaxPooling3D(pool_size=(3,3,3))(conv4_6)
flat4 = Flatten()(pool4_3)

merge = concatenate([flat1, flat2, flat3, flat4])

hidden = Dense(128, activation = 'relu')(merge)
dropout = Dropout(0.5)(hidden)
output = Dense(3, activation = 'softmax')(dropout)

model = Model(inputs = input_shape, outputs = output)

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = Adam(lr = 0.000001),
    metrics = ['accuracy']
)

In [ ]:
timee = time.time()

history = model.fit([train_flair, train_t1, train_t1ce, train_t2], train_labels, batch_size = 20, epochs = 20)

print(time.time()-timee)

In [ ]:
model.save('/content/drive/My Drive/Upenn_Data/Models/2class_model.h5')

In [ ]:
results = model.evaluate([val_flair, val_t1, val_t1ce, val_t2], val_labels, batch_size =5)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])
plt.title('Training')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper left')
plt.show()